In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [2]:
ruta = "C:/Users/9 ----- SIG/Downloads/documentos.xlsx"
df = pd.read_excel(ruta)

In [5]:
# Crear navegador
driver = webdriver.Chrome()

# Abrir página de inicio de sesión
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(0.5)

# Ingresar usuario y contraseña
usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
usuario.send_keys("1826-admin")

contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
contrasena.send_keys("1826Uan9123")

# Clic en botón ingresar
boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
boton_ingresar.click()

# Ir a MDM -> Novedad Poblacional
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_129"))).click()
time.sleep(0.5)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_135"))).click()
time.sleep(0.5)

# Seleccionar tipo de documento (una sola vez)
dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "filter:tipoDoc_label")))
dropdown.click()
time.sleep(0.5)
opcion = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[@data-label='Cédula de ciudadanía']")))
opcion.click()
time.sleep(0.5)

# Lista donde guardaremos los resultados
resultados = []

# Procesar cada cédula del DataFrame
for i in range(len(df)):
    cedula = str(df['NUM_DOCUMENTO'].iloc[i])
    try:
        # Ingresar número de cédula
        input_cedula = driver.find_element(By.ID, "filter:numDoc")
        input_cedula.clear()
        input_cedula.send_keys(cedula)
        time.sleep(0.3)

        # Clic en buscar
        boton_buscar = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']"))
        )
        boton_buscar.click()
        time.sleep(0.5)

        # Buscar la tabla de resultados (programas asociados a la cédula)
        try:
            tabla = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.ID, "datos:j_idt120_data"))
            )
            filas = tabla.find_elements(By.TAG_NAME, "tr")

            programas = []
            for fila in filas:
                columnas = fila.find_elements(By.TAG_NAME, "td")
                if len(columnas) >= 3:
                    codigo = columnas[1].text.strip()   # Columna 2 = Código Programa
                    nombre = columnas[2].text.strip()   # Columna 3 = Nombre Programa
                    programas.append((codigo, nombre))

            if programas:
                for codigo, nombre in programas:
                    resultados.append({
                        "cedula": cedula,
                        "codigo_programa": codigo,
                        "nombre_programa": nombre
                    })
            else:
                resultados.append({
                    "cedula": cedula,
                    "codigo_programa": "No encontrado",
                    "nombre_programa": "No encontrado"
                })

        except:
            # Si no existe tabla → no hay resultados
            resultados.append({
                "cedula": cedula,
                "codigo_programa": "No encontrado",
                "nombre_programa": "No encontrado"
            })

    except Exception as e:
        print(f"Error procesando cédula {cedula}: {e}")
        resultados.append({
            "cedula": cedula,
            "codigo_programa": "Error",
            "nombre_programa": "Error"
        })

# Cerrar navegador
driver.quit()

# Convertir resultados a DataFrame y exportar a Excel
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("resultados.xlsx", index=False)

print("✅ Proceso completado. Archivo 'resultados.xlsx' generado.")

✅ Proceso completado. Archivo 'resultados.xlsx' generado.
